In [67]:
import json
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
import spacy
from tqdm import tqdm
import scml
from scml import nlp as snlp
import mylib

In [68]:
tqdm.pandas()
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
scml.seed_everything()

In [69]:
#%%time
#nlp = spacy.load("pretrained/spacy/en_core_web_lg/en_core_web_lg-2.3.1")
#nlp.max_length = 1_000_000
#nlp.path

In [70]:
%%time
#pretrained_dir = "pretrained/bert-large-uncased-whole-word-masking-finetuned-squad"
#pretrained_dir = "pretrained/distilbert-base-cased-distilled-squad"
pretrained_dir = "pretrained/mrm8488/electra-small-finetuned-squadv2"
from_tf = False
tokenizer = AutoTokenizer.from_pretrained(pretrained_dir)
model = AutoModelForQuestionAnswering.from_pretrained(pretrained_dir, from_tf=from_tf)
print(repr(model.config))

ElectraConfig {
  "_name_or_path": "pretrained/mrm8488/electra-small-finetuned-squadv2",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.5.1",
  "type_vocab_size": 2,
  "vocab_size": 30522
}

CPU times: user 1.56 s, sys: 31.2 ms, total: 1.59 s
Wall time: 637 ms


In [71]:
train = pd.read_parquet("input/train.parquet")
train = train.sample(1000)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 11503 to 7243
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            1000 non-null   object
 1   ground_truth  1000 non-null   object
 2   is_multi      1000 non-null   int8  
dtypes: int8(1), object(2)
memory usage: 24.4+ KB


In [72]:
train["PredictionString"] = train.progress_apply(
    mylib.qa_predict(
        data_dir=f"input/train",
        model=model,
        tokenizer=tokenizer,
        questions=["name study"],
        #question="name dataset study survey program initiative",
        #question="name of dataset database study survey program initiative model assessment archive collection catalog registry",
        max_window=4,
    ),
    axis=1,
)

100%|██████████| 1000/1000 [27:47<00:00,  1.67s/it] 


In [73]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 11503 to 7243
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Id                1000 non-null   object
 1   ground_truth      1000 non-null   object
 2   is_multi          1000 non-null   int8  
 3   PredictionString  1000 non-null   object
dtypes: int8(1), object(3)
memory usage: 32.2+ KB


In [74]:
train.head(20)

,Id,ground_truth,is_multi,PredictionString
11503,72897d3a-7abb-486f-94da-139bfadbe40d,agricultural resource management survey,0,
2776,bf81c91c-7ef7-4e17-86d5-128baccdf0bc,early childhood longitudinal study,0,
1508,f3c6ab46-7e14-4f07-b316-ee2b708eb2e3,common core of data|nces common core of data,1,
1399,a3c6594b-854d-4ec2-b30f-7ff5ffe4ab09,ibtracs|international best track archive for climate stewardship,1,
3968,000efc17-13d8-433d-8f62-a3932fe4f3b8,adni|alzheimer s disease neuroimaging initiative adni,1,
8644,12c6c6aa-32c6-43a9-afe4-97e5b0a5c305,rural urban continuum codes,0,
7177,bd3f64b7-cf15-405e-873c-eef81646322e,trends in international mathematics and science study,0,
13975,68971ea4-459c-4e52-b20e-a353cfca16c5,genome sequence of sars cov 2|genome sequences of sars cov 2,1,
9684,3346172e-844e-44c3-861f-9050c2157511,early childhood longitudinal study,0,
5684,c9f1465b-0e69-4eb0-979b-33bf3d9dd9ac,adni,0,


In [75]:
train.to_parquet("output/validation.parquet", index=False)

In [76]:
%%time
score = mylib.fbeta(y_true=train["ground_truth"], y_pred=train["PredictionString"])
print(f"score={score:.4f}")

score=0.0430
CPU times: user 15.6 ms, sys: 31.2 ms, total: 46.9 ms
Wall time: 14.1 ms


In [77]:
# 0.1746
# question="name dataset study survey program initiative",
# window_length=4,
# stride=3,
# n_window=6,

# 0.1711
# questions=["what study", "what survey"],
# n_window=4,

In [78]:
assert "|".join([]) == ""
print("hello world"[:100])

hello world
